In [1]:
!pip install nengo --quiet
!pip install nengo-dl --quiet
!pip install nengo-loihi --quiet
!pip install nengo-extras==0.5.0 --quiet
!pip install nengo-gui --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.3/535.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 843.7/843.7 kB 5.9 MB/s eta 0:00:00


In [2]:
# Setup the environment
import nengo
from nengo.dists import Uniform
from nengo.processes import Piecewise

In [3]:
# Create the network object to which we can add ensembles, connections, etc.
model = nengo.Network(label="Scalar Addition")

with model:
    # Inputs to drive the activity in ensembles A and B
    stim_A = nengo.Node(
        Piecewise({0: -0.75, 1.25: 0.5, 2.5: 0.70, 3.75: 0}), label="Input A"
    )
    stim_B = nengo.Node(
        Piecewise({0: 0.25, 1.25: -0.5, 2.5: 0.85, 3.75: 0}), label="Input B"
    )

    # Ensembles with 100 LIF neurons each
    # Represents the first input (A)
    ens_A = nengo.Ensemble(100, dimensions=1, max_rates=Uniform(100, 200), label="A")
    # Represents the second input (B)
    ens_B = nengo.Ensemble(100, dimensions=1, max_rates=Uniform(100, 200), label="B")
    # Represents the sum of two inputs
    ens_sum = nengo.Ensemble(
        100, dimensions=1, max_rates=Uniform(100, 200), radius=2, label="Sum"
    )

    # Connecting the input nodes to ensembles
    nengo.Connection(stim_A, ens_A)
    nengo.Connection(stim_B, ens_B)

    # Connecting ensembles A and B to the Sum ensemble
    nengo.Connection(ens_A, ens_sum)
    nengo.Connection(ens_B, ens_sum)

In [4]:
with model:
    input_A_probe = nengo.Probe(stim_A)  # Input to ensemble A
    input_B_probe = nengo.Probe(stim_B)  # Input to ensemble B
    A_probe = nengo.Probe(ens_A, synapse=0.01)  # Decoded output of ensemble A
    B_probe = nengo.Probe(ens_B, synapse=0.01)  # Decoded output of ensemble B
    sum_probe = nengo.Probe(ens_sum, synapse=0.01)  # Decoded output of Sum ensemble

In [ ]:
from nengo_gui.ipython import IPythonViz

IPythonViz(model, "ch3-addition.py.cfg")